In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 30 17:12:58 2020

@author: hisham
"""

import chardet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import requests
import json
from flask import jsonify
from csv import writer
from statistics import mean
import itertools
import psycopg2

COLLECTING DATA FROM API

In [4]:
url = 'http://coronavirus-tracker-api.herokuapp.com/all'
url_iso2 = 'https://restcountries.eu/rest/v2/'


response_iso2 = requests.get(url_iso2)
lst_iso2 = json.loads(response_iso2.text)

response = requests.get(url)
lst = json.loads(response.text)

dict_pop = {}
dict_iso3 = {}
dict_affected = {}
dict_deaths = {}
dict_recovered = {}

for i in lst_iso2:
    dict_pop[i['alpha2Code']] = i['population']
    dict_iso3[i['alpha2Code']] = i['alpha3Code']
    dict_affected[i['alpha2Code']] = {}
    dict_deaths[i['alpha2Code']] = {}
    dict_recovered[i['alpha2Code']] = {}
      

## INITIALIZATION ##

for i in lst['confirmed']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = 0

for i in lst['recovered']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = 0

for i in lst['deaths']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = 0         


## Summing up ##

for i in lst['deaths']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = dict_deaths[code][j] + i['history'][j] 

for i in lst['recovered']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = dict_recovered[code][j] + i['history'][j] 

for i in lst['confirmed']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = dict_affected[code][j] + i['history'][j]                         



with open('country_model_new.csv', 'w') as csv_file:
    csv_writer = writer(csv_file)
    headers = ['Country_name', 'Country_code_iso2', 'Country_code_iso3', 'Date', 'Tot_affected', 'Tot_dead', 'Tot_recov', 'Population']
    csv_writer.writerow(headers)
    for i in lst['confirmed']['locations']:
        
        name = i['country']
        code = i['country_code']
        if code in dict_iso3:
            #prevDate = ''
            for j in i['history']:
                entry = [name, code, dict_iso3[code], j, dict_affected[code][j], dict_deaths[code][j], dict_recovered[code][j], dict_pop[code]]
                #entry = [name, code, dict_iso3[code], j, dict_affected[code][j], dict_deaths[code][j], dict_recovered[code].get(j,prevDate), dict_pop[code]]
                #s = j.split('/')
                #prevDate = s[0] + '/' + str(int(s[1]) - 1) + '/' + s[2] 
                csv_writer.writerow(entry)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

MAIN CODE


In [2]:
with open('country_model_new.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

dataset = pd.read_csv('country_model_new.csv', keep_default_na=False,  encoding=result['encoding'])
X = dataset.iloc[:, :].values

#%%
#----------fixing date (Y-m-d)---------
from datetime import datetime as dt, timedelta
for i in range(len(X)):
    # print(i)
    yesterday_str = X[i][3]
    yesterday = dt.strptime(yesterday_str, '%m/%d/%y').date()
    yesterday_str = yesterday.strftime('%Y-%m-%d')
    X[i][3] = yesterday_str

temp1 = np.array(np.ones((len(X),1)))
temp1 = X[:,4] - (X[:,5] + X[:,6])
temp1 = temp1.reshape((len(X),1))
X = np.hstack((X[:, :7], temp1, X[:, 7:]))

#%%
#------delteing extra data----------
# from extra, comes extra new datas -_-
# now let's delete repeating datas
country_data = (X == "Bangladesh").sum()
# country_data = 105

extra = country_data * 230
X = np.delete(X, slice(extra, len(X)), 0)

flag = []
i = 0
while i < len(X):
    j = i+country_data
    while j < len(X):
        if X[i][0] == X[j][0] and flag.count(j) == 0:
            flag.append(j)
        j = j + country_data
    i = i + country_data 
   
flag.reverse()
for i in flag:
    X = np.delete(X, slice(i, i+country_data), 0)
# total country should be 161    
#%%
# 3rd part : calculation
codeList = []
i = 0
while i < len(X):
    #40 threshold set as total cases to avoid unexpected results
    if X[i+country_data-1][4] >= 40 and X[i][0] != "Serbia" and X[i][0]!= "Antigua and Barbuda":
        codeList.append(X[i][1])
    i = i + country_data

# code = codeList[80]     #to test
codeIndex = 1
report = np.empty((0, 8), object)
idx = 0
#--------------looping starts-----------
# code = 'IN'
for code in itertools.islice(codeList, 0, len(codeList)):
    datas = 0
    # arr: country data by dates
    # datas: number of countries
    arr = np.empty((country_data, 9), dtype=object)
    for row in X:
        if row[codeIndex] == code : 
            arr[datas]=row
            datas = datas+ 1
    print(arr[0][0])
     
    arr = arr[np.argsort(arr[:,3])]
    # init: previous values (0 for dead, 1 for affected)
    init = np.empty((datas, 2), dtype=object)
    init[0][0] = arr[0][5]
    init[0][1] = arr[0][4]
    for i in range(1, datas):
        init[i][0] = arr[i][5] - arr[i-1][5]
        init[i][1] = arr[i][4] - arr[i-1][4]
    # for last 2 equations of SIRD
    def calLamda(D):
        lst = []
        for i in range(len(arr) - 1):
            l = 0
            if D == True:
                if arr[i][7] != 0 :     #divide by zero!
                    l = (arr[i+1][5] - arr[i][5]) / arr[i][7]       #lamdaD=(D1 - D0)/ Id0
            else:
                if arr[i][7] != 0 :     #divide by zero!
                    l = (arr[i+1][6] - arr[i][6]) / arr[i][7]       #lamdaR=(R1 - R0)/ Ir0
            if arr[i][3] >= "2020-03-05": 
                
                if l<0:
                    l = 0  #timing considered from March 05 to see its growing shape
                lst.append(l)
        return lst
    ##ektu chnage
    """"lamdaD = calLamda(True)
    lamdaR = calLamda(False)
    # last 5 values average taken as constant
    ld = mean(lamdaD[-5:])
    # print(ld)
    lr = mean(lamdaR[-5:])
    # print(lr)    
    
    lstBSbyL = []
    for i in range(len(arr) - 1):
        #BS0 = ((Id1 + Ir1 + Id0*lamdaD + Ir0*lamdaR) - (Id0 + Ir0)) / (Id0 + Ir0)
        l = 0
        fplus = arr[i+1][7] + arr[i][7]*(ld + lr)
        splus = arr[i][7]
        if splus != 0:
            l = (fplus - splus)/ splus    
        if arr[i][3] >= "2020-03-05":                           #timing considered from March 05 to see its growing shape
            lstBSbyL.append(l)            
    # last 5 values average taken as constant
    lbsl = mean(lstBSbyL[-5:])
    # print(lbsl/ (ld + lr))
    
    # pop: population of corresponding country
    for i in range(len(X)):
        if X[i][codeIndex] == code:
            pop = X[i][8]
            # print(pop)
            break
    # number of confirmed cases subtracted from actual population
    pop = pop - arr[-1][4]
    # now some hardcoding max, min, average bullshit begins!!
    minConst = 50
    maxConst = 1000
    midConst = 300
    pop1 = pop / minConst
    beta1 = (lbsl / pop1)
    
    pop3 = pop / maxConst
    beta3 = (lbsl / pop3)
    
    pop2 = pop / midConst
    beta2 = (lbsl / pop2)"""
    
    ##hishu hospital theke dise
    lamdaD = calLamda(True)
    lamdaR = calLamda(False)
    days_to_take = 8
    # last 'days_to_take' values average taken as constant
    ld = mean(lamdaD[-days_to_take:])
    # print(ld)
    lr = mean(lamdaR[-days_to_take:])
    # print(lr)    
    lstBSbyL = []
    for i in range(len(arr) - 1):
        #BS0 = ((Id1 + Ir1 + Id0*lamdaD + Ir0*lamdaR) - (Id0 + Ir0)) / (Id0 + Ir0)
        l = 0
        fplus = arr[i+1][7] + arr[i][7]*(ld + lr)
        splus = arr[i][7]
        if splus != 0:
            l = (fplus - splus)/ splus    
        if arr[i][3] >= "2020-03-05":                           #timing considered from March 05 to see its growing shape
            lstBSbyL.append(l)            
    # last 5 values average taken as con'stant
    lbsl = mean(lstBSbyL[-days_to_take:])
    # print(lbsl/ (ld + lr))
    # pop: population of corresponding country
    for i in range(len(X)):
        if X[i][codeIndex] == code:
            pop = X[i][8]
            # print(pop)
            break
    # number of confirmed cases subtracted from actual population
    pop = pop - arr[-1][4]
    # now some hardcoding max, min, average bullshit begins!!
    maxconst = 800
    midconst = 200
    minconst = 100
    pop1 = pop / minconst
    beta1 = (lbsl / pop1)
    pop3 = pop / maxconst
    beta3 = (lbsl / pop3)
    pop2 = pop / midconst
    beta2 = (lbsl / pop2)
    
    # here goes our incremental SIRD model; (Eliza, try an ODE version? I haven't tried)
    # this function returns future daily affecteds or deads
    def calcDiff(pop, beta):
        print("pop,beta,array[-1][7]" )
        print(pop,beta,arr[-1][7])
        final = np.empty((30, 4), dtype=object)
        final[0][0] = pop - beta*pop*arr[-1][7]                             #S
        final[0][1] = arr[-1][7] + beta*pop*arr[-1][7] - (ld+lr)*arr[-1][7] #I
        final[0][2] = arr[-1][6] + lr*arr[-1][7]                            #R
        final[0][3] = arr[-1][5] + ld*arr[-1][7]                            #D

        for i in range(1, len(final)):
            final[i][0] = final[i-1][0] - beta*final[i-1][0]*final[i-1][1]
            final[i][1] = final[i-1][1] + beta*final[i-1][0]*final[i-1][1] - (ld+lr)*final[i-1][1]
            final[i][2] = final[i-1][2] + lr*final[i-1][1]
            final[i][3] = final[i-1][3] + ld*final[i-1][1]
            print(final[i][0],final[i][1],final[i][2],final[i][3])
            
        final[:, [0,1,2,3]] = final[:, [0,1,2,3]].astype(int)    
        final = np.append(final, np.ones((len(final),1)), axis = 1)
        final[:,4] = final[:,1] + final[:,2] + final[:,3]                   #cases=active+R+D
        
        diffDead=[]
        diffDead.append(abs(final[0][3] - arr[-1][5]))
        for i in range(1,len(final)):
            diffDead.append(final[i][3] - final[i-1][3])        
        diffAffec=[]
        diffAffec.append(abs(final[0][4] - arr[-1][4]))
        for i in range(1,len(final)):
            diffAffec.append(final[i][4] - final[i-1][4])
       
        return (diffAffec, diffDead)
    
    dA1, dD1 = calcDiff(pop1, beta1)
    dA2, dD2 = calcDiff(pop2, beta2)
    dA3, dD3 = calcDiff(pop3, beta3)
    
    """    
    #ploting stuffs
    # init[:,0]-> dead, 1-> affected; change dA-> for dead
    prevVals = init[:,1]
    x = np.arange(datas, datas+30) 
    plt.plot(prevVals, color = 'b')
    plt.plot(x,dA1, '--', color = 'r')    
    plt.plot(x,dA2, '--', color = 'black')
    plt.plot(x,dA3, '--', color = 'r')
    plt.title(f"{arr[0][0]}")
    plt.show()
    """  
    # now lets create our report, which will be pushed into database
    name = arr[0][0]
    name = np.full((30,1), name)
    code2 = arr[0][1]
    code2 = np.full((30,1), code2)
    
    one_day = timedelta(days = 1)
    yesterday_str = arr[-1][3]
    yesterday = dt.strptime(yesterday_str, '%Y-%m-%d').date()
    Today = np.full((30,1), yesterday_str)
    
    day_predict = []
    for i in range(30):
        day = yesterday + one_day
        yesterday = day
        day = day.strftime("%Y-%m-%d")
        day_predict.append(day)
    day_predict = np.array(day_predict)
    day_predict = day_predict.reshape((30,1))
    
    def createReport (string, v1, v2, v3):
        Type = np.full((30,1), string)        
        v1 = np.array(v1)
        v2 = np.array(v2)
        v3 = np.array(v3)
        v1 = v1.reshape((30,1))
        v2 = v2.reshape((30,1))
        v3 = v3.reshape((30,1))   
        values = np.hstack((Type, name, code2, Today, day_predict, v2, v1, v3))
        values = values.reshape((30, 8))
        return values
    
    reportAffec = createReport("Affected", dA1, dA2, dA3)
    reportDead = createReport("Dead", dD1, dD2, dD3)
    report = np.vstack((report[:idx, :], reportAffec, reportDead, report[idx+60:, :]))
    idx = idx + 60
    
   
    
"""
report
0-> type (affected/ dead)
1-> country_name
2-> code2
3-> database pushing date
4-> prediction date
5-> middle val
6-> max val
7-> min val
"""

Afghanistan
pop,beta,array[-1][7]
276204.35 3.106982095768686e-08 9916
276034.2644354987 9897.588405060173 25689.45555032192 1293.0416091191844
275949.37939992064 9888.356211279086 25779.058275580468 1297.5561132197718
275864.59962191427 9879.106549846883 25868.577421921706 1302.0664063171353
275779.9251701906 9869.839518154306 25958.012831211236 1306.5724804439014
275695.3561125577 9860.555213568838 26047.36434619634 1311.074327677119
275610.89251592295 9851.253733433005 26136.631810505773 1315.571940138248
275526.53444629465 9841.935175062687 26225.81506864952 1320.0653099931485
275442.28196878394 9832.59963574542 26314.913966018576 1324.5544294520682
275358.13514760695 9823.247212738726 26403.928348884667 1329.0392907696303
275274.0940460867 9813.878003268434 26492.858064399996 1333.519886244819
275190.158726655 9804.492104527015 26581.70296059695 1337.996208220965
275106.3292508545 9795.089613671922 26670.462886387802 1342.468249085731
275022.60567934054 9785.670627823936 26759.137

Andorra
pop,beta,array[-1][7]
770.89 9.097031169589006e-05 66
761.3768398541474 74.3715977454925 808.14156240036 52.0
756.2256630507488 78.89885416806833 808.7654827811829 52.0
750.7978886275029 83.66472801863223 809.4273833538649 52.0
745.0835611027086 88.6771729625862 810.1292659347051 52.0
739.0729794179092 93.94382151696608 810.8731990651246 52.0
732.756788770601 99.47189591191949 811.6613153174793 52.0
726.1260803737591 105.26811177317117 812.4958078530694 52.0
719.1724988395067 111.33857498837425 813.3789261721188 52.0
711.8883566823037 117.68867231915401 814.312970998542 52.0
704.2667552100116 124.32295654295591 815.3002882470321 52.0
696.30171082132 131.2450271521042 816.3432620265754 52.0
687.98828545895 138.4574078979249 817.4443066431246 52.0
679.3227196850921 145.96142273994684 818.6058575749606 52.0
670.3025665557366 153.7570720368987 819.8303614073643 52.0
660.9268241828555 161.84291108851795 821.1202647286261 52.0
651.196064598486 170.21593339439886 822.4780020071147 52.

Armenia
pop,beta,array[-1][7]
29553.5 9.27201558053735e-07 8546
29091.633402373933 8206.681165641437 30538.976818096107 766.2086138885215
28870.267961796453 8039.484484531425 30921.6179572919 772.1295963802231
28655.062559619608 7874.044034197068 31296.46345690594 777.9299492773856
28445.85693681303 7710.436921327052 31663.595202188935 783.6109396709882
28242.49383245128 7548.733603731094 32023.09867353444 789.1738902831895
28044.819038567246 7388.998150291332 32375.062636471153 794.6201746702737
27852.681442546156 7231.288501065169 32719.578843775533 799.951212613146
27665.933058120892 7075.656726471711 33056.74174971191 805.1684656954891
27484.429045968347 6922.149284605458 33386.64823635728 810.2734330689176
27308.027724846714 6770.807275826003 33709.39735192351 815.2676474037802
27136.5905741552 6621.666693871259 34025.0900609499 820.1526710236442
26969.982228740944 6474.758672833994 34333.82900620411 824.9300922209536
26808.07046672313 6330.109729427566 34635.71828209843 829.60152

pop,beta,array[-1][7]
87046.27 8.760708124646929e-07 1602
86801.08724102218 1628.7635427446996 19200.89785496388 719.5213612692445
86677.22971064689 1642.0498288168385 19310.698934391086 720.2915261451932
86552.54001476422 1655.2663194028676 19421.395692358492 721.0679734744238
86427.02753583272 1668.4083702235687 19532.983423689744 721.8506702539678
86300.70200027738 1681.471308317676 19645.457110120144 722.6395812847977
86173.57347736027 1694.450435118365 19758.81141836309 723.4346691582649
86045.65237775812 1707.341029613974 19873.040698383913 724.2358942439915
85916.94945184382 1720.1383515901407 19988.138981886783 725.043214679258
85787.47578766957 1732.8376449503237 20104.099981020183 725.8565863599223
85657.24280864939 1745.4341411114608 20220.91708730623 726.6759629329106
85526.26227093946 1757.923062471311 20338.583370798904 727.5012957903126
85394.54626051501 1770.299625943812 20457.09157947606 728.3325340651171
85262.1071899431 1782.5590465585885 20576.4341388697 729.1696246

Bahrain
pop,beta,array[-1][7]
13633.64 8.005490036445423e-06 2723
13053.613536430035 2570.847243301639 39396.144063841835 149.03515642649256
12784.95852613832 2489.112099985207 39745.56027725453 150.00909662194636
12530.198255916392 2404.6221975902536 40083.86747423119 150.9520722621686
12288.989696030374 2318.096022019508 40410.69124221635 151.86303973377375
12060.93683601097 2230.2071148451823 40725.75482152916 152.74122761469462
11845.602065738269 2141.5788178509956 41028.872996763275 153.5861196474648
11642.516469149621 2052.780821236699 41319.94527389216 154.3974357215225
11451.188983683662 1964.3273165442342 41598.94858822427 155.17511154783548
11271.114424823912 1876.676536228762 41865.92976131355 155.9192776337805
11101.780410451758 1790.2314561738551 42120.9978952908 156.63023808359347
10942.673245231568 1705.341442426186 42364.31686267729 157.30844966496082
10793.282842285262 1622.3046362013015 42596.09802001398 157.95450149945992
10653.106769427688 1541.3708892674204 42816.5

...

318664.0583973361 263504.8914447457 1459.221758631198 46699.283399286986
317898.2202198213 264208.85355493485 1461.419777825307 46758.96144741853
317132.18151877465 264912.8508846773 1463.623669101037 46818.79892744701
316365.9525108466 265616.87320905237 1465.8334327521711 46878.795847348876
315599.54342204105 266320.9102961869 1468.0490689872013 46938.95221278486
314832.9644871647 267024.95190780767 1470.2705779292708 46999.268027098406
314066.2259492753 267728.9877997944 1472.4979596161206 47059.743291314204
313299.3380591294 268433.0077227337 1474.7312140000404 47120.37800413689
312532.3110746288 269137.00142247364 1476.9703409478248 47181.17216194982
311765.1552602662 269840.9586406791 1479.2153402407337 47242.12575881403
310997.8808865708 270544.86911538756 1481.4662115744575 47303.23878646724
310230.4982295522 271248.7225815657 1483.7229545590862 47364.511234323094
309463.0175701446 271952.5087716659 1485.985568719085 47425.943089470406
308695.44919365016 272656.21741618396 1488

2936756.503626503 3193291.9634441608 1571302.6377152603 159394.34521407646
2874222.081355685 3228245.131380526 1597716.6938890896 160561.54337470006
2812349.333936445 3262234.7252886374 1624419.8733099925 161741.51746492556
2751171.084434406 3295236.2447306337 1651404.205551587 162933.91528337388
2690718.241572489 3327227.3152277097 1678661.5175264291 164138.37567337262
2631019.761993099 3358187.7076354977 1706183.4510713764 165354.52930002747
2572102.6225834545 3388099.347288951 1733961.4806932202 166581.9994343754
2513991.8026408185 3416946.3132283404 1761986.9313899616 167820.4027408806
2456710.2755816886 3444714.8278879793 1790250.9964656858 169069.35006464744
2400279.009833282 3471393.237691712 1818744.7552601427 170328.44721486454
2344716.97848735 3496971.985053674 1847459.190717821 171597.29574115615
2290041.177245819 3521443.5723290653 1876385.2067254176 172875.4936996994
2236266.6501451475 3544802.51829757 1905513.645151119 174162.636406165
2183406.522511664 3567045.307791661 

Venezuela
pop,beta,array[-1][7]
310084.94 3.159661606590286e-07 8628
308356.2267385247 9471.01147213512 12277.76310092079 185.93868841939707
307433.4648304125 9919.011395748143 12746.125600317297 192.33817352208948
306469.9462323349 10385.31071071426 13236.642688436472 199.04036851438596
305464.29368132504 10870.369369808144 13750.219310721239 206.05763814557406
304415.1249794783 11374.629204272209 14287.783158298207 213.40265795127476
303321.05765391357 11898.510172249826 14850.283771827757 221.08840200887118
302180.7140763945 12442.406335616637 15438.691459830836 229.12812815803412
300992.7270543189 13006.68156711715 16053.996018061149 237.53536050282545
299755.74590119394 13591.664993207205 16697.205236585738 246.32386901314885
298468.4429904766 14197.646181972425 17369.343181504035 255.50764604696622
297129.52079173404 14824.870089946146 18071.448238698817 265.10087962103006
295737.719382451 15473.531786585747 18804.570907696107 275.11792326722184
294291.8244224674 16143.7709805696

131817.93326888554 9465.884225664631 4657.4021786708345 342.2503267790379
131049.02819741517 9937.58364657034 4933.932939123463 362.9252168910563
130246.51600231664 10428.079981099638 5224.243648332961 384.63036825079706
129409.55059735826 10937.629138295355 5528.8834282508105 407.4068360956102
128537.32955171212 11466.435336877525 5848.408876804379 431.29623460601607
127629.10186294101 12014.644955206311 6183.382557846311 456.3406240064012
126684.17609271112 12582.340211615445 6534.371311440354 482.5823842331238
125701.92881497907 13169.532730693265 6901.9443795281795 510.0640747995341
pop,beta,array[-1][7]
71181.235 1.232443544728187e-06 2835
70670.73224772944 3162.4646098678854 1186.3052301135165 82.73291228916335
70395.28889258734 3338.6142984748344 1278.6916065600221 89.64020237780534
70105.63666087194 3523.4421935611967 1376.2239161473435 96.93222941952993
69801.206888564 3717.2444846534822 1479.1556788826883 104.62794789984042
69481.42704039643 3920.311932898197 1587.74906747945

'\nreport\n0-> type (affected/ dead)\n1-> country_name\n2-> code2\n3-> database pushing date\n4-> prediction date\n5-> middle val\n6-> max val\n7-> min val\n'

In [3]:
noCountries = int(len(report)/60)
noCountries




156

In [4]:
report[1][3]


'2020-08-02'

In [5]:
len(report)

9360

In [6]:
report[10][6]

'83'

FUNCTION FOR SELECTING REGIONID

In [7]:
def getRegion(regID):
    try:
        

       
    # Print PostgreSQL Connection properties
        print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
        cursor.execute("SELECT version();")
        record = cursor.fetchone()
        print("You are connected to - ", record,"\n")
    
        postgres_select_query = "select id from regions where alpha2 = %s"
       
        cursor.execute(postgres_select_query, (regID,))
        
        
        reg = cursor.fetchall()
        for r in reg :
            print(r[0])
            return r[0]
        #print(reg)

       # connection.commit()
       # count = cursor.rowcount
       # print (count, "Record inserted successfully into worldprediction table")

    except (Exception, psycopg2.Error) as error :
        if(connection):
            print("Error fetching data from PostgreSQL table", error)
    
    #return reg[0]

FUNCTION FOR INSERTING

In [8]:
def insertInfo2(regid,alpha2,typ,ins_date,pred_date,median,maxi,mini,name):
    try:
        postgres_insert_query = """INSERT INTO longtermprediction(region_id,alpha2,type,insertion_date,prediction_date,median,maximum,minimum,name)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        cursor.execute(postgres_insert_query, (regid,alpha2,typ,ins_date,pred_date,median,maxi,mini,name))

        connection.commit()
        count = cursor.rowcount
        print (count, "Record inserted successfully into countryprediction table")
        
    except (Exception, psycopg2.Error) as error :
        if(connection):
            print("Error inserting data in PostgreSQL table", error)

In [9]:
# res =[]

# for i in range(len(report)):
#     regid = getRegion(report[i][2])
#     res.append((regid,report[i][2],report[i][0],report[i][3],report[i][4],report[i][5],report[i][6],report[i][7],report[i][1]))
    
#     print(i)

INSERTING, MIGHT CHANGE LATER WITH SQL SCRIPTS

In [10]:
len(res)

NameError: name 'res' is not defined

In [11]:
# # import psycopg2
# connection = psycopg2.connect(user = "teamcoronosis",
#                                   password = "p1jGQwEMANAGZ5b72xWh",
#                                   host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
#                                   port = "5432",
#                                   database = "postgres")
# cursor = connection.cursor()

# insertInfo3(res)
# #closing connection
# if(connection):
#     cursor.close()
#     connection.close()
#     print("PostgreSQL connection is closed")

In [12]:
for k in range(len(report)):
    if report[k][1] == "Cote d'Ivoire" :
        
        report[k][1] = "Cote dIvoire"
      

In [13]:
with open("data.psql","w") as sql_file:
    for j in range(len(report)):
        sql_file.write("INSERT INTO longtermprediction(region_id,alpha2,type,insertion_date,prediction_date,median,maximum,minimum,name)VALUES((select id from regions where alpha2 = '%s'LIMIT 1),'%s','%s','%s','%s',%s,%s,%s,'%s');\n" % (report[j][2],report[j][2],report[j][0],report[j][3],report[j][4],report[j][5],report[j][6],report[j][7],report[j][1]))
        

In [95]:
connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
cursor = connection.cursor()
with open('data.psql', 'r') as f:
#sql_file = open('data.sql', 'r')
    cursor.execute(f.readline())
# sql_file = open("1.sql")
# sql_as_string = sql_file.read()
#with connection as cursor:
#cursor.execute(open("1.sql", "r").read())
#cursor.execute(open("1.sql", "r").read())

if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")


PostgreSQL connection is closed


In [ ]:
connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
cursor = connection.cursor()
cursor.execute("select * from longtermprediction")
reg = cursor.fetchall()
for r in reg:
    print(r)
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

In [1]:
def deleteData(date):
    try:
        connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
        cursor = connection.cursor()

        # Update single record now
        sql_delete_query = """Delete from longtermprediction where insertion_date = %s"""
        cursor.execute(sql_delete_query, (date, ))
        connection.commit()
        count = cursor.rowcount
        print(count, "Record deleted successfully ")

    except (Exception, psycopg2.Error) as error:
        print("Error in Delete operation", error)

    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [5]:
deleteData('2020/07/10')

9240 Record deleted successfully 
PostgreSQL connection is closed
